In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("manjilkarki/deepfake-and-real-images")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/deepfake-and-real-images


In [3]:
#importing packages

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.utils import image_dataset_from_directory
import warnings

warnings.filterwarnings('ignore')

In [30]:
#checking for duplications

import os
def get_all_image_paths(folder_path):
    image_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.jpg')):
                image_paths.append(os.path.join(root, file))
    return set(image_paths)

train_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Train')
val_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Validation')
test_paths = get_all_image_paths('/kaggle/input/deepfake-and-real-images/Dataset/Test')

# Intersections
print("Train ∩ Val:", len(train_paths & val_paths))
print("Train ∩ Test:", len(train_paths & test_paths))
print("Val ∩ Test:", len(val_paths & test_paths))

Train ∩ Val: 0
Train ∩ Test: 0
Val ∩ Test: 0


In [4]:
#loading image data
def load_1stimage_data(path):
    img_size = (256, 256)
    batch_size = 64

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Train",
        label_mode="binary",  
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Validation",
        label_mode="binary",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True  
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path + "/Dataset/Test",
        label_mode="binary",
        image_size=img_size,
        batch_size=batch_size,
        shuffle=False  
    )


    return train_ds,val_ds ,test_ds

train,val,test=load_1stimage_data('/kaggle/input/deepfake-and-real-images')


Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [31]:
print("Class names and their labels:", train.class_names)

Class names and their labels: ['Fake', 'Real']


In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

model = Sequential([
    Input((256, 256, 3)),
    Rescaling(1./255),
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1),
    RandomContrast(0.1),

    Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(256, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    Conv2D(512, (3,3), padding='same', kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'),

    GlobalAveragePooling2D(),

    Dense(512, kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128, kernel_regularizer=l2(1e-4)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])


In [54]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)              │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_flip_4 (RandomFlip)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_4 (RandomRotation)   │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom_4 (RandomZoom)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_translation_4                 │ (None, 256, 256, 3)         │               0 │
│ (RandomTranslation)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_contrast_4 (RandomContrast)   │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_58 (Conv2D)                   │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_66               │ (None, 256, 256, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_66 (LeakyReLU)           │ (None, 256, 256, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_58 (MaxPooling2D)      │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_59 (Conv2D)                   │ (None, 128, 128, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_67               │ (None, 128, 128, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_67 (LeakyReLU)           │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_59 (MaxPooling2D)      │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_60 (Conv2D)                   │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_68               │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_68 (LeakyReLU)           │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_60 (MaxPooling2D)      │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_61 (Conv2D)                   │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 10,356,609 (39.51 MB)

 Trainable params: 10,348,865 (39.48 MB)

 Non-trainable params: 7,744 (30.25 KB)

In [55]:
from tensorflow.keras.optimizers import Adam

custom_adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [56]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint=ModelCheckpoint(filepath='best_model.keras',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')

earlystop=EarlyStopping(monitor='val_accuracy',patience=5,verbose=1,restore_best_weights=True)

lr_scheduler=ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


model.fit(train,epochs=10,validation_data=val,callbacks=[checkpoint, earlystop, lr_scheduler])


Epoch 1/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.7705 - loss: 0.6370
Epoch 1: val_accuracy improved from -inf to 0.78432, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 469s 207ms/step - accuracy: 0.7706 - loss: 0.6369 - val_accuracy: 0.7843 - val_loss: 0.6968 - learning_rate: 0.0010
Epoch 2/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.9329 - loss: 0.2583
Epoch 2: val_accuracy did not improve from 0.78432
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 454s 208ms/step - accuracy: 0.9329 - loss: 0.2582 - val_accuracy: 0.7081 - val_loss: 0.7737 - learning_rate: 0.0010
Epoch 3/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accuracy: 0.9409 - loss: 0.2296
Epoch 3: val_accuracy improved from 0.78432 to 0.86058, saving model to best_model.keras
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 455s 208ms/step - accuracy: 0.9409 - loss: 0.2296 - val_accuracy: 0.8606 - val_loss: 0.5089 - learning_rate: 0.0010
Epoch 4/10
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - accur

In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

y_true = []
y_pred = []

for images, labels in test:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend((preds > 0.5).astype("int32").flatten())  

y_true = np.array(y_true)
y_pred = np.array(y_pred)

test_loss, test_accuracy = model.evaluate(test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Precision: {precision_score(y_true, y_pred):.4f}")
print(f"Recall: {recall_score(y_true, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━

In [58]:
!zip -r folder.zip /kaggle/working/best_model.keras

updating: kaggle/working/best_model.keras (deflated 8%)


In [59]:
from IPython.display import FileLink

FileLink(r'folder.zip')

/kaggle/working/folder.zip